In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
# 1) GPU 환경용 패키지 설치 (한 번만 실행)
!pip install --upgrade torch torchvision torchaudio \
  --extra-index-url https://download.pytorch.org/whl/cu118 --quiet
!pip install pytorch-tcn scikit-learn --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 130.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 94.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 55.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# 2) 라이브러리 임포트 테스트
import torch
print("CUDA available:", torch.cuda.is_available())
from pytorch_tcn import TemporalConv1d
# ──────────────────────────────────────────

CUDA available: True


In [ ]:
# 2) 라이브러리 임포트
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from pytorch_tcn import TemporalConv1d



In [ ]:
# 3) CSV 로컬 복사 (한 번만 실행)
# Colab 런타임에서만 한 번 실행해주세요.
!cp /content/drive/MyDrive/Capstone/GrayScale/features_tcn.csv /content/

# 3) 데이터 로드 및 전처리
df = pd.read_csv('/content/TCN_features.csv')  # git-hub에서는 파일 용량 제한 때문에 T/F를 분리하여 올려 놓은 상태
df = df.dropna(subset=['T/F']).reset_index(drop=True)

# 3-1) X, y 생성
feat_cols = [f'feat_{i}' for i in range(308)]
X = df[feat_cols].values.reshape(-1, 44, 7)
y = df['T/F'].astype(int).values

# 3-2) 특성별 표준화
scaler = StandardScaler().fit(X.reshape(-1, 7))
X_scaled = scaler.transform(X.reshape(-1, 7)).reshape(-1, 44, 7)

# 3-3) 학습/검증 분할
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=42, stratify=y
)



In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split

# y 는 이미 df['T/F'].astype(int).values 로 정의되어 있으므로 그대로 사용
indices = np.arange(len(df))

# 인덱스와 y를 함께 split
train_idx, test_idx, _, _ = train_test_split(
    indices, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

# test 세트에 해당하는 파일명 추출
test_filenames = df.loc[test_idx, '파일명'].values
print(test_filenames)


In [ ]:
from collections import Counter
import pandas as pd
import numpy as np

def print_data_distribution(y, y_train, y_test):
    total_counter = Counter(y)
    train_counter = Counter(y_train)
    test_counter  = Counter(y_test)

    print("📦 전체 데이터")
    print(f"총 개수: {len(y)}")
    for k in sorted(total_counter.keys()):
        print(f"  클래스 {k}: {total_counter[k]}개 ({(total_counter[k]/len(y))*100:.1f}%)")
    print()

    print("🧠 학습 데이터")
    print(f"총 개수: {len(y_train)}")
    for k in sorted(train_counter.keys()):
        print(f"  클래스 {k}: {train_counter[k]}개 ({(train_counter[k]/len(y_train))*100:.1f}%)")
    print()

    print("🧪 테스트 데이터")
    print(f"총 개수: {len(y_test)}")
    for k in sorted(test_counter.keys()):
        print(f"  클래스 {k}: {test_counter[k]}개 ({(test_counter[k]/len(y_test))*100:.1f}%)")

# 사용 예시
print_data_distribution(y, y_train, y_test)


📦 전체 데이터
총 개수: 30610
  클래스 0: 15327개 (50.1%)
  클래스 1: 15283개 (49.9%)

🧠 학습 데이터
총 개수: 24488
  클래스 0: 12262개 (50.1%)
  클래스 1: 12226개 (49.9%)

🧪 테스트 데이터
총 개수: 6122
  클래스 0: 3065개 (50.1%)
  클래스 1: 3057개 (49.9%)


In [ ]:
# 4) Dataset & DataLoader
class SequenceDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.tensor(X, dtype=torch.float).permute(0,2,1)  # (B, C, L)
        self.y = torch.tensor(y, dtype=torch.long)
    def __len__(self):
        return len(self.y)
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

train_loader = DataLoader(SequenceDataset(X_train, y_train), batch_size=32, shuffle=True)
test_loader  = DataLoader(SequenceDataset(X_test,  y_test),  batch_size=32)



In [ ]:
# 5) TCNModel 정의 (채널·깊이·드롭아웃)
import torch.nn as nn
from pytorch_tcn import TemporalConv1d

class TCNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.tcn1 = TemporalConv1d(7,  64, kernel_size=3, dilation=1, padding=0)
        self.tcn2 = TemporalConv1d(64, 64, kernel_size=3, dilation=2, padding=0)
        self.tcn3 = TemporalConv1d(64, 64, kernel_size=3, dilation=4, padding=0)
        self.tcn4 = TemporalConv1d(64, 64, kernel_size=3, dilation=8, padding=0)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.3)
        self.fc   = nn.Linear(64, 1)

    def forward(self, x):
        x = self.relu(self.tcn1(x)); x = self.dropout(x)
        x = self.relu(self.tcn2(x)); x = self.dropout(x)
        x = self.relu(self.tcn3(x)); x = self.dropout(x)
        x = self.relu(self.tcn4(x)); x = self.dropout(x)
        out = x[:, :, -1]                   # 마지막 타임스텝
        return self.fc(out).squeeze(-1)



In [ ]:
# 6) 디바이스 설정: GPU 고정
device = 'cuda'
model = TCNModel().to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=15, gamma=0.5)



In [ ]:
# 7) 학습 루프
n_epochs = 65
for epoch in range(1, n_epochs+1):
    model.train()
    losses = []
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device).float()
        optimizer.zero_grad()
        loss = criterion(model(xb), yb)
        loss.backward()
        optimizer.step()
        losses.append(loss.item())
    scheduler.step()
    print(f"[Epoch {epoch:02d}/{n_epochs}] Train Loss: {np.mean(losses):.4f}")



[Epoch 01/65] Train Loss: 0.2542
[Epoch 02/65] Train Loss: 0.2043
[Epoch 03/65] Train Loss: 0.1956
[Epoch 04/65] Train Loss: 0.1903
[Epoch 05/65] Train Loss: 0.1844
[Epoch 06/65] Train Loss: 0.1779
[Epoch 07/65] Train Loss: 0.1771
[Epoch 08/65] Train Loss: 0.1720
[Epoch 09/65] Train Loss: 0.1696
[Epoch 10/65] Train Loss: 0.1654
[Epoch 11/65] Train Loss: 0.1627
[Epoch 12/65] Train Loss: 0.1605
[Epoch 13/65] Train Loss: 0.1604
[Epoch 14/65] Train Loss: 0.1578
[Epoch 15/65] Train Loss: 0.1513
[Epoch 16/65] Train Loss: 0.1471
[Epoch 17/65] Train Loss: 0.1429
[Epoch 18/65] Train Loss: 0.1445
[Epoch 19/65] Train Loss: 0.1409
[Epoch 20/65] Train Loss: 0.1396
[Epoch 21/65] Train Loss: 0.1387
[Epoch 22/65] Train Loss: 0.1394
[Epoch 23/65] Train Loss: 0.1367
[Epoch 24/65] Train Loss: 0.1376
[Epoch 25/65] Train Loss: 0.1324
[Epoch 26/65] Train Loss: 0.1325
[Epoch 27/65] Train Loss: 0.1343
[Epoch 28/65] Train Loss: 0.1320
[Epoch 29/65] Train Loss: 0.1335
[Epoch 30/65] Train Loss: 0.1305
[Epoch 31/

In [ ]:
# 8) 평가
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb = xb.to(device)
        preds = (torch.sigmoid(model(xb)) > 0.5).cpu().long().numpy()
        y_pred.extend(preds)
        y_true.extend(yb.numpy())

print(f"\nTest Accuracy: {accuracy_score(y_true, y_pred):.4f}")
print("Confusion Matrix:\n", confusion_matrix(y_true, y_pred))



Test Accuracy: 0.9499
Confusion Matrix:
 [[2914  151]
 [ 156 2901]]


In [ ]:
# 학습이 끝난 후
save_path = "tcn_model_state.pth"
torch.save(model.state_dict(), save_path)
print(f"모델 가중치를 '{save_path}'로 저장했습니다.")


모델 가중치를 'tcn_model_state.pth'로 저장했습니다.


In [ ]:
from joblib import dump

# 이미 fit() 한 scaler
dump(scaler, "scaler.pkl")


['scaler.pkl']

In [ ]:
from joblib import load

scaler = load("scaler.pkl")


In [ ]:
# inference.py
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 1) 모델 인스턴스 생성
model = TCNModel().to(device)

# 2) 저장된 가중치 불러오기
checkpoint = torch.load("tcn_model_state.pth", map_location=device)
model.load_state_dict(checkpoint)

# 3) 평가 모드 전환
model.eval()

for i in range(20):
  # 1) NumPy → Tensor, float 타입으로
  sample_np = X_test[i]                   # shape: (44, 7)
  sample = torch.tensor(sample_np, dtype=torch.float)

  # 2) (44,7) → (7,44)로 차원 순서 변경
  sample = sample.permute(1, 0)           # shape: (7, 44)

  # 3) 배치 차원 추가 → (1,7,44)
  sample = sample.unsqueeze(0)            # shape: (1, 7, 44)

  # 4) 예측 실행
  with torch.no_grad():
      sample = sample.to(device)
      logits = model(sample)
      prob   = torch.sigmoid(logits)
      pred_label = (prob > 0.5).long()
      print("예측:",end="")
      print(pred_label.item(),end="")
      print(", 정답:",end="")
      print(y_test[i])



예측:1, 정답:1
예측:0, 정답:0
예측:1, 정답:1
예측:1, 정답:1
예측:1, 정답:1
예측:1, 정답:1
예측:1, 정답:1
예측:1, 정답:1
예측:1, 정답:1
예측:1, 정답:1
예측:0, 정답:0
예측:1, 정답:1
예측:1, 정답:1
예측:0, 정답:0
예측:1, 정답:1
예측:0, 정답:0
예측:1, 정답:1
예측:0, 정답:0
예측:0, 정답:0
예측:1, 정답:0
